In [33]:
import os
import json
from geomet import wkt
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
from tqdm import tqdm
from pathlib import Path
import tifffile as tiff
import numpy as np
import PIL
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image

In [2]:
def createdirectory():
    pathstring = "./Dataset"
    for i in ["/Image","/Label"]:
        for j in ["/Pre","/Post"]:
            for k in ["/Tier1","/Tier3","/Hold","/Test"]:
                Path(pathstring+i+j+k).mkdir(parents=True, exist_ok=True)
                print(pathstring+i+j+k)

In [3]:
inputfol = ["./Dataset/xview2_geotiff/geotiffs/hold/labels/","./Dataset/xview2_geotiff/geotiffs/test/labels/","./Dataset/xview2_geotiff/geotiffs/tier1/labels/","./Dataset/xview2_geotiff/geotiffs/tier3/labels/"]
outputpostfol = ["./Dataset/Validation/Post/Label512/","./Dataset/Test/Post/Label512/","./Dataset/Tier1/Post/Label512/","./Dataset/Tier3/Post/Label512/"]
outputprefol = ["./Dataset/Validation/Pre/Label512/","./Dataset/Test/Pre/Label512/","./Dataset/Tier1/Pre/Label512/","./Dataset/Tier3/Pre/Label512/"]

In [21]:
def createlabels(size):
    geopath = "./Dataset/xview2_geotiff/geotiffs"
    outputpath = "./Dataset/Label"
    inputpath = ["/hold/labels/","/test/labels/","/tier1/labels/","/tier3/labels/"]
    outputpost = ["/Post/Hold/","/Post/Test/","/Post/Tier1/","/Post/Tier3/"]
    outputpre = ["/Pre/Hold/","/Pre/Test/","/Pre/Tier1/","/Pre/Tier3/"]
    json_list = []
    for i in range(4):
        for filename in os.listdir(geopath+inputpath[i]):
            if filename.endswith('.json'):
                data = open(geopath + inputpath[i] + filename)
                json_list.append((json.load(data),filename))
        pre = outputpath + outputpre[i]
        post = outputpath + outputpost[i]
        with tqdm(total=len(json_list), desc='Saving to ' + pre + ' and ' + post, unit='img') as pbar:
            for js in json_list:
                width = js[0]['metadata']['width']
                height = js[0]['metadata']['height']
                image = Image.new("L", (width, height), color = (0))
                draw = ImageDraw.Draw(image)
                if('pre_disaster' in js[1]):
                    for j in js[0]['features']['xy']:
                        polygonlist = wkt.loads(j['wkt'])
                        polygontuples = [(x,y) for x,y in polygonlist['coordinates'][0]]
                        draw.polygon((polygontuples), fill=(1))
                    image = image.resize(size,Image.NEAREST)
                    image.save(pre+js[1].split('.')[0] + '.png')
                if('post_disaster' in js[1]):
                    for j in js[0]['features']['xy']:
                        subtype = j['properties']['subtype']
                        fill = (0)
                        polygonlist = wkt.loads(j['wkt'])
                        polygontuples = [(x,y) for x,y in polygonlist['coordinates'][0]]
                        match subtype:
                            case "no-damage":
                                fill = (1)
                            case "minor-damage":
                                fill = (2)
                            case "major-damage":
                                fill = (3)
                            case "destroyed":
                                fill = (4)
                            case _:
                                fill = (0)
                        draw.polygon((polygontuples), fill=fill)
                    image = image.resize(size,Image.NEAREST)
                    image.save(post+js[1].split('.')[0] + '.png')
                pbar.update(1)

In [22]:
createlabels((512,512))

Saving to ./Dataset/Label/Pre/Hold/ and ./Dataset/Label/Post/Hold/: 100%|█████████| 1866/1866 [00:25<00:00, 72.48img/s]
Saving to ./Dataset/Label/Pre/Test/ and ./Dataset/Label/Post/Test/: 100%|█████████| 3732/3732 [00:49<00:00, 75.75img/s]
Saving to ./Dataset/Label/Pre/Tier1/ and ./Dataset/Label/Post/Tier1/: 100%|███████| 9330/9330 [02:02<00:00, 76.37img/s]
Saving to ./Dataset/Label/Pre/Tier3/ and ./Dataset/Label/Post/Tier3/: 100%|████| 22068/22068 [03:22<00:00, 108.83img/s]


In [40]:
def createpngs(size):
    geopath = "./Dataset/xview2_geotiff/geotiffs"
    outputpath = "./Dataset/Image"
    inputpath = ["/hold/images/","/test/images/","/tier1/images/","/tier3/images/"]
    outputpost = ["/Post/Hold/","/Post/Test/","/Post/Tier1/","/Post/Tier3/"]
    outputpre = ["/Pre/Hold/","/Pre/Test/","/Pre/Tier1/","/Pre/Tier3/"]
    for i in range(4):
        img_list = []
        for filename in os.listdir(geopath+inputpath[i]):
            img_list.append(filename)
        pre = outputpath + outputpre[i]
        post = outputpath + outputpost[i]
        with tqdm(total=len(img_list), desc='Saving to ' + pre + ' and ' + post, unit='img') as pbar:
            for img in img_list:
                if('pre_disaster' in img):           
                    im = tiff.imread(geopath + inputpath[i] + img)
                    ima = im.astype(dtype=np.uint8)
                    imarray = Image.fromarray(ima)
                    imarray = imarray.resize(size,PIL.Image.NEAREST)
                    imarray.save(pre + img.split(".")[0] + ".png", "PNG", quality=100)
                if('post_disaster' in img):
                    im = tiff.imread(geopath + inputpath[i] + img)
                    ima = im.astype(dtype=np.uint8)
                    imarray = Image.fromarray(ima)
                    imarray = imarray.resize(size,PIL.Image.NEAREST)
                    imarray.save(post + img.split(".")[0] + ".png", "PNG", quality=100)
                pbar.update(1)

In [41]:
createpngs((512,512))

Saving to ./Dataset/Image/Pre/Hold/ and ./Dataset/Image/Post/Hold/: 100%|█████████| 1866/1866 [02:49<00:00, 11.00img/s]
Saving to ./Dataset/Image/Pre/Test/ and ./Dataset/Image/Post/Test/: 100%|█████████| 1866/1866 [02:43<00:00, 11.43img/s]
Saving to ./Dataset/Image/Pre/Tier1/ and ./Dataset/Image/Post/Tier1/: 100%|███████| 5598/5598 [08:12<00:00, 11.37img/s]
Saving to ./Dataset/Image/Pre/Tier3/ and ./Dataset/Image/Post/Tier3/: 100%|█████| 12738/12738 [19:14<00:00, 11.03img/s]
